# Pneumonia_Chest_X-Ray

CNN image detection of pneumonia from chest xrays.  
Project By: [Uzair Bin Asim](https://github.com/Uzair05)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os 
import zipfile

## Download dataset
Make sure to navigate to the proper directory.

In [ ]:
#!pip install --upgrade kaggle

## Remeber to navigate to proper directory.
!cd "/content/drive/MyDrive/nih_xray" && kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [8]:
os.chdir("/content/drive/MyDrive/nih_xray") ## Remeber to navigate to proper directory.
print(os.getcwd())

/content/drive/MyDrive/nih_xray


In [ ]:
try:
    if not os.path.isdir("./data"):
        os.mkdir("./data")
except Exception as err:
    print(f"Error in creating data/:\t{err}")

if os.path.isfile("./chest-xray-pneumonia.zip"):
    localzip = "./chest-xray-pneumonia.zip"
    with zipfile.ZipFile(localzip, 'r') as zip_ref:
        zip_ref.extractall('data/')

In [ ]:
## Clean the dataspace
!rm -rf data/chest_xray/__MACOSX && cd data/ && find -type f | grep -E "\.DS_Store" | xargs -I{} rm {}

## Handle Data
Create the train, test, and validation datasets.

In [ ]:
#!pip install -q --upgrade tensorflow==2.7

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf ## Using tensorflow==2.7
import matplotlib.pyplot as plt

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
main_path = "data/chest_xray/chest_xray"

test_dir, train_dir = os.path.join(main_path, "test"), os.path.join(main_path, "train")
validation_dir = os.path.join(main_path, "val") 

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0, 
    rotation_range=10, 
    width_shift_range=0.3, 
    height_shift_range=0.3, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True, 
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)
val_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

In [9]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=60
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=20
)
val_generator = val_datagen.flow_from_directory(
    validation_dir, 
    target_size=(150,150), 
    class_mode='binary', 
    batch_size=20
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


## Create Model - v2

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, padding='same', input_shape=(150,150, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, padding='same'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, padding='same'),
        tf.keras.layers.Flatten(), 
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])
    model.compile(
        #optimizer='adam', 
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), 
        #loss='binary_crossentropy', 
        loss=tf.keras.losses.BinaryCrossentropy(), 
        metrics=['accuracy']
)
model.summary()

## Train Model - v1

In [11]:
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        accuracy = 0.90 # Percentage Accuracy.
        if(logs.get('accuracy') != None) and (logs.get('accuracy') >= accuracy): # Experiment with changing this value
          print(f"\nReached {accuracy*100}% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [ ]:
## Using Google Colab TPU

train_generator_tpu = tf.data.Dataset.from_tensor_slices(train_generator)
test_generator_tpu = tf.data.Dataset.from_tensor_slices(test_generator)

history = model.fit(
    train_generator_tpu, 
    validation_data = test_generator_tpu,
    steps_per_epoch = 85, 
    epochs = 15,
    validation_steps = 30,
    verbose = 1, 
    #callbacks=[callbacks]
)

## Evaluation

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
print(f"Evaluation accuracy with validation dataset:\t{model.evaluate(val_generator)[-1]*100}%")

## Save Model

In [ ]:
#tf.keras.models.save_model(model, filepath="./my_model/OwnCNN_EfficientNetB3_99p.h5", overwrite=True, save_format='h5')